# Retrieval augmented chat - Mistral-7B-Instruct

This notebook is the primary demonstration of the project with the baseline model. Here we'll bring up the baseline model and vector database and start asking questions both with and without the vector database.

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

## Initialize some variables

In [2]:
collection_name = "Corpus"
model_dir = "mistralai/Mistral-7B-Instruct-v0.1"
device_map = {"": 0}
device = "cuda"
database_top_n_results = 2

## Load shared code
This file defines the `ChatModel` and `Retrieval` classes used below.

In [3]:
%run shared_code.ipynb

## Load the model

In [4]:
language_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Load the model into the ChatModel class from `shared_code.ipynb`

In [5]:
chat_model = ChatModel(language_model, tokenizer)

## Trying the model without access to the vector database.

In [6]:
chat_model.basic_chat("How do I loop a GIF?")

To loop a GIF in HTML, you can use the `animation` property. This is an example of how to create an animation using CSS:
```
@keyframes fadeIn {
  from { opacity: 0; }
  to   { opacity: 1; }
}

body {
  background-image: url('https://example.com/your_gif_file.gif');
  background-size: cover;
  animation: fadeIn 5s linear infinite;
}
```
This code will make your body's background image a GIF that slowly fades in (over 5 seconds) and repeats infinitely over time. You can adjust the duration, timing function, delay, fill mode, animation name, etc. as per your requirement.


## Load the collection into the RetrievalAugmentedChat class from `shared_code.ipynb`

In [7]:
rac = RetrievalAugmentedChat("db/", collection_name, database_top_n_results, chat_model)

## Run retrieval augmented chat
Notice that the responses have switched from a general HTML/JavaScript context to include the document contents which provide examples of encoding the gif with ImageMagick so that it loops by default.

In [8]:
rac.markdown_chat("How do I loop a GIF?")

To loop a GIF, you need to specify that it should repeat itself continuously. There are two ways to achieve this: one way is to set the loop count manually while playing the video; the other way is to automate the process using command line tools or software.

Setting the Loop Count Manually:
You can use a media player application such as Windows Media Player, VLC, ffmpeg, etc., to play back your GIF files and control its loop settings. In these applications, you typically have options to select how many times you want the video to repeat before stopping. Here's an example using ffmpeg:
```javascript
$ ffmpeg -i input_file.gif -loop 0 output_file.gif
```
In this command, replace "input\_file" with the path to your GIF file and "output\_file" with the desired name of the looped output file. "-loop 0" specifies that the video should not loop, so you would change this value to a positive integer if you wanted the video to play repeatedly at specified intervals. For instance, to loop the video five times:
```css
$ ffmpeg -i input_file.gif -loop 5 output_file.gif
```
Automating Looping Process:
Another method involves automating the process using scripting languages or specific tools designed to handle image processing tasks. Here we will demonstrate using Python and various libraries:

Using Pillow (Python Imaging Library):
Pillow is a popular open-source library for handling images in Python. It provides several functions you can use to manipulate, resize, save, display images, and more. First, you need to download and install the Pillow package:
```
pip install pillow
```
Then, let's write a simple Python script to loop through all the frame files in a directory containing BMP images named "frame\_*.jpg", and combine them into a single image. This loop will run infinitely, creating new animations every time you restart the script.
```python
import os
from PIL import Image
import time

while True:
    # Get current working directory
    dirpath = os.getcwd()
    
    # Create a list of names from files ending with .jpg in the current directory
    filenames = [fn for fn in os.listdir(dirpath) if fn.endswith(".jpg")]

    # Open each image individually and stack them up
    img_stack = []
    for filename in reversed(filenames):
        filepath = os.path.join(dirpath, filename)
        with Image.open(filepath) as img:
            # If the image has alpha channel (transparency), strip it off
            if img.mode == 'RGB':
                img = Image.new("RGBA", (img.size))
                img.putdata(img.load())
            else:
                img = img.resize((int(img.width / 2), int(img.height / 2)))
            img_stack.append(img)

    # Combine all the stacks to form a single composite image
    img_layer = Image.merge('firstover', img_stack, 1)

    # Save the output in the current directory
    outputfilename = "output" + "_".join([time.strftime("%Y-%m-%d_%H%M%S", time.localtime()), ".png"])
    img_layer.save(os.path.join(dirpath, outputfilename))
    print("Image saved: ", outputfilename)

    # Wait for a minute before moving onto next iteration
    time.sleep(60)
```
This code assumes you have RGB images (without transparency) in your directory and will generate a series of "outputXXX.png" files where X represents timestamp information to indicate the date and time when

 **Reference documents:** 

* [corpus/imagemagick/set-a-gif-to-loop.md](corpus/imagemagick/set-a-gif-to-loop.md) distance: 0.97
* [corpus/imagemagick/compress-animated-gif.md](corpus/imagemagick/compress-animated-gif.md) distance: 1.28

**Inference time in seconds 17.8272**


In [9]:
rac.markdown_chat("Can I use npx with GitHub actions?")

Yes, you can use npx with GitHub actions. In fact, npx by default caches its dependencies, making it well suited for running scripts in environments where speed and memory usage are critical.

When using npx, you need to ensure that your npm configuration files contain the necessary permissions to access packages stored in private registries or other systems that enforces authentication for certain operations such as installations or publishing. You may also need to consider updating your project dependencies and adjusting registry settings accordingly if migrating away from using traditional package.json dependencies.

It should be noted that while GitHub actions caches the dependency tree of a CLI command executed via npx, any changes made directly within the npx command execution environment (such as modifying package.json or locally installed dependencies) are not tracked and therefore not included in the cache. Therefore, for scenarios where you require persistent management of these changes across multiple executions, additional measures might be required.

 **Reference documents:** 

* [corpus/github-actions/npm-cache-with-npx-no-package.md](corpus/github-actions/npm-cache-with-npx-no-package.md) distance: 0.88
* [corpus/github-actions/attach-generated-file-to-release.md](corpus/github-actions/attach-generated-file-to-release.md) distance: 1.11

**Inference time in seconds 4.1823**
